In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-09 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-09 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-09 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-09 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-09 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [56]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (10, 3, 5)]
num_layers = [1]
ffn_hidden_dim_sizes = [[32,32], [128,128], [512,512]]
dropout_rates = [0.1]
learning_rates = [5e-4, 3e-4, 1e-4, 1e-5]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [57]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [58]:
size = 5

## umap

In [59]:
(
    swmhau_network_umap_kfold_5,
    best_swmhau_network_umap_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold_best_model.csv


In [60]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2582605/2619527520.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.00001        0.624010   
                                                                                      0.00010        0.640294   
                                                                                      0.00030        0.624649   
                                                                                      0.00050        0.630460   
                                                          (128, 128)     0.1          0.00001        0.646296   
                                                                                      0.00010        0.629183   
                                                                                      0.00030        0.624266   
                                                                                      0.00050        0.628544   
                                                          (512, 512)     0.1          0.00001        0.636909   
                                                                                      0.00010        0.625990   
                                                                                      0.00030        0.635568   
                                                                                      0.00050        0.635377   
           12              3         10        1          (32, 32)       0.1          0.00001        0.606577   
                                                                                      0.00010        0.628289   
                                                                                      0.00030        0.631481   
                                                                                      0.00050        0.643295   
                                                          (128, 128)     0.1          0.00001        0.623946   
                                                                                      0.00010        0.619540   
                                                                                      0.00030        0.614176   
                                                                                      0.00050        0.629757   
                                                          (512, 512)     0.1          0.00001        0.635057   
                                                                                      0.00010        0.618455   
                                                                                      0.00030        0.622797   
                                                                                      0.00050        0.630651   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.00001        0.292018   
                                                                                      0.00010        0.358926   
                                                                                      0.00030        0.479885   
                                                                                      0.00050        0.489851   
                                                          (128, 128)     0.1          0.00001        0.296808   
                                                                                      0.00010        0.451741   
                                                                                      0.00030        0.483085   
                                                                                      0.00050        0.493772   
                                

In [61]:
best_swmhau_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.640996,0.505014,"[0.7728579643473261, 0.3967168262653898, 0.345...",0.504270,"[0.7684391080617495, 0.4434250764525994, 0.300...",0.513898,"[0.7773279352226721, 0.3589108910891089, 0.405...",None,0.672447,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.618582,0.504578,"[0.7410590943975441, 0.4588492352512746, 0.313...",0.498157,"[0.7896630683676807, 0.4106910039113429, 0.294...",0.518086,"[0.6980913823019086, 0.5198019801980198, 0.336...",None,0.646114,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.629694,0.501158,"[0.7637864936596874, 0.4063545150501673, 0.333...",0.495246,"[0.7791817087845969, 0.411864406779661, 0.2946...",0.511205,"[0.7489878542510121, 0.400990099009901, 0.3836...",None,0.659281,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [62]:
best_swmhau_network_umap_kfold_5["f1"].mean()

0.50358348521465

In [63]:
best_swmhau_network_umap_kfold_5["precision"].mean()

0.49922438069026037

In [64]:
best_swmhau_network_umap_kfold_5["recall"].mean()

0.5143960763919075

In [65]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.75923452, 0.42064019, 0.33087575])

In [66]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.77909463, 0.4219935 , 0.29658502])

In [67]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.74146906, 0.42656766, 0.37515152])

# w=11

In [24]:
size = 11

## umap

In [25]:
(
    swmhau_network_umap_kfold_11,
    best_swmhau_network_umap_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold_best_model.csv


In [26]:
swmhau_network_umap_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2582605/404807304.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.624840   
                                                                                      0.0003         0.617369   
                                                                                      0.0005         0.627395   
                                                          (128, 128)     0.1          0.0001         0.616858   
                                                                                      0.0003         0.624521   
                                                                                      0.0005         0.627650   
                                                          (512, 512)     0.1          0.0001         0.612899   
                                                                                      0.0003         0.611814   
                                                                                      0.0005         0.633142   
           12              3         10        1          (32, 32)       0.1          0.0001         0.614623   
                                                                                      0.0003         0.616284   
                                                                                      0.0005         0.617050   
                                                          (128, 128)     0.1          0.0001         0.623308   
                                                                                      0.0003         0.620434   
                                                                                      0.0005         0.627139   
                                                          (512, 512)     0.1          0.0001         0.614751   
                                                                                      0.0003         0.626309   
                                                                                      0.0005         0.632120   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.375354   
                                                                                      0.0003         0.450537   
                                                                                      0.0005         0.482031   
                                                          (128, 128)     0.1          0.0001         0.436992   
                                                                                      0.0003         0.474755   
                                                                                      0.0005         0.509560   
                                                          (512, 512)     0.1          0.0001         0.465875   
                                                                                      0.0003         0.487970   
                                                                                      0.0005         0.503375   
           12              3         10        1          (32, 32)       0.1          0.0001         0.365546   
                                                                                      0.0003         0.422264   
                                                                                      0.0005         0.483115   
                                                          (128, 128)     0.1          0.0001         0.380603   
                                                                                      0.0003         0.462209   
                                

In [27]:
best_swmhau_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.631992,0.508518,"[0.7619188628960617, 0.42144638403990026, 0.34...",0.501755,"[0.7806432038834952, 0.42462311557788945, 0.3]",0.520190,"[0.7440717177559283, 0.4183168316831683, 0.398...",None,0.664258,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.618774,0.489979,"[0.7500373524577917, 0.4262295081967213, 0.293...",0.484783,"[0.7758887171561051, 0.40444444444444444, 0.27...",0.497571,"[0.7258530942741469, 0.4504950495049505, 0.316...",None,0.661368,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.630651,0.492020,"[0.7653105545099175, 0.39547038327526124, 0.31...",0.489657,"[0.7663090750942302, 0.4188191881918819, 0.283...",0.497816,"[0.7643146327356853, 0.37458745874587457, 0.35...",None,0.655588,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [28]:
best_swmhau_network_umap_kfold_11["f1"].mean()

0.49683892577972094

In [29]:
best_swmhau_network_umap_kfold_11["precision"].mean()

0.49206514301548526

In [30]:
best_swmhau_network_umap_kfold_11["recall"].mean()

0.5051921881989626

In [31]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75908892, 0.41438209, 0.31704576])

In [32]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.77428033, 0.41596225, 0.28595285])

In [33]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.74474648, 0.41446645, 0.35636364])

# w=20

In [34]:
size = 20

## umap

In [35]:
(
    swmhau_network_umap_kfold_20,
    best_swmhau_network_umap_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold_best_model.csv


In [36]:
swmhau_network_umap_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2582605/1154838213.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.616411   
                                                                                      0.0003         0.600639   
                                                                                      0.0005         0.620626   
                                                          (128, 128)     0.1          0.0001         0.609706   
                                                                                      0.0003         0.609962   
                                                                                      0.0005         0.617816   
                                                          (512, 512)     0.1          0.0001         0.626565   
                                                                                      0.0003         0.603640   
                                                                                      0.0005         0.615517   
           12              3         10        1          (32, 32)       0.1          0.0001         0.593934   
                                                                                      0.0003         0.611047   
                                                                                      0.0005         0.630332   
                                                          (128, 128)     0.1          0.0001         0.594253   
                                                                                      0.0003         0.604981   
                                                                                      0.0005         0.628736   
                                                          (512, 512)     0.1          0.0001         0.614304   
                                                                                      0.0003         0.614176   
                                                                                      0.0005         0.616092   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.394254   
                                                                                      0.0003         0.430100   
                                                                                      0.0005         0.475865   
                                                          (128, 128)     0.1          0.0001         0.411819   
                                                                                      0.0003         0.460985   
                                                                                      0.0005         0.482887   
                                                          (512, 512)     0.1          0.0001         0.425627   
                                                                                      0.0003         0.478064   
                                                                                      0.0005         0.488827   
           12              3         10        1          (32, 32)       0.1          0.0001         0.386219   
                                                                                      0.0003         0.425751   
                                                                                      0.0005         0.473099   
                                                          (128, 128)     0.1          0.0001         0.393337   
                                                                                      0.0003         0.454608   
                                

In [37]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.622989,0.506070,"[0.7535148070595273, 0.4116666666666667, 0.353...",0.497816,"[0.780359355638166, 0.4158249158249158, 0.2972...",0.523531,"[0.7284557547715442, 0.4075907590759076, 0.434...",None,0.662331,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.615709,0.488078,"[0.7509120093389756, 0.3595905651980418, 0.353...",0.482739,"[0.7578792341678939, 0.3903381642512077, 0.3]",0.502771,"[0.7440717177559283, 0.3333333333333333, 0.430...",None,0.653821,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.609579,0.495265,"[0.7470101195952161, 0.411741053478086, 0.3270...",0.487405,"[0.7950391644908616, 0.4015686274509804, 0.265...",0.517450,"[0.7044534412955465, 0.42244224422442245, 0.42...",None,0.658960,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [38]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.4964709616411047

In [39]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.4893200453209174

In [40]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.5145840379295303

In [41]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.75047898, 0.39433276, 0.34460114])

In [42]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.77775925, 0.40257724, 0.28762365])

In [43]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.7256603 , 0.38778878, 0.43030303])

# w=35

In [44]:
size = 35

## umap

In [45]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_35_kfold_best_model.csv


In [46]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2582605/220877647.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.594828   
                                                                                      0.0003         0.602490   
                                                                                      0.0005         0.612197   
                                                          (128, 128)     0.1          0.0001         0.614432   
                                                                                      0.0003         0.602874   
                                                                                      0.0005         0.626564   
                                                          (512, 512)     0.1          0.0001         0.604278   
                                                                                      0.0003         0.613793   
                                                                                      0.0005         0.632184   
           12              3         10        1          (32, 32)       0.1          0.0001         0.576309   
                                                                                      0.0003         0.586973   
                                                                                      0.0005         0.613346   
                                                          (128, 128)     0.1          0.0001         0.579247   
                                                                                      0.0003         0.591890   
                                                                                      0.0005         0.609962   
                                                          (512, 512)     0.1          0.0001         0.587739   
                                                                                      0.0003         0.606130   
                                                                                      0.0005         0.629630   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.406044   
                                                                                      0.0003         0.430566   
                                                                                      0.0005         0.477059   
                                                          (128, 128)     0.1          0.0001         0.412622   
                                                                                      0.0003         0.450956   
                                                                                      0.0005         0.488067   
                                                          (512, 512)     0.1          0.0001         0.452499   
                                                                                      0.0003         0.472238   
                                                                                      0.0005         0.503830   
           12              3         10        1          (32, 32)       0.1          0.0001         0.397054   
                                                                                      0.0003         0.415893   
                                                                                      0.0005         0.471974   
                                                          (128, 128)     0.1          0.0001         0.406277   
                                                                                      0.0003         0.452300   
                                

In [47]:
best_swmhau_network_umap_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.641188,0.502772,"[0.7726683004180481, 0.40438079191238413, 0.33...",0.502129,"[0.7703363035354988, 0.41308089500860584, 0.32...",0.503685,"[0.7750144592249856, 0.39603960396039606, 0.34]",None,0.682402,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.598276,0.471043,"[0.7375110848359444, 0.35175031632222686, 0.32...",0.464746,"[0.7542321644498187, 0.359792924935289, 0.2802...",0.483070,"[0.7215153267784846, 0.34405940594059403, 0.38...",None,0.676140,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.640230,0.490387,"[0.7794138670478913, 0.379965457685665, 0.3117...",0.491078,"[0.7707096409386486, 0.39855072463768115, 0.30...",0.490451,"[0.788316946211683, 0.36303630363036304, 0.32]",None,0.677746,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [48]:
best_swmhau_network_umap_kfold_35["f1"].mean()

0.488067195483414

In [49]:
best_swmhau_network_umap_kfold_35["precision"].mean()

0.4859842380096248

In [50]:
best_swmhau_network_umap_kfold_35["recall"].mean()

0.4924020454869855

In [51]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

array([0.76319775, 0.37869886, 0.32230498])

In [52]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

array([0.7650927 , 0.39047485, 0.30238516])

In [53]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

array([0.76161558, 0.36771177, 0.34787879])

# w=80

In [54]:
size = 80

## umap

In [55]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)

## w=110

In [ ]:
size = 110

## umap

In [ ]:
(
    swmhau_network_umap_kfold_110,
    best_swmhau_network_umap_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_20

In [ ]:
best_swmhau_network_umap_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)